We should've used classes rather than functions to define our architecture.

The residual connections were also missed before -- both improvements below

In [71]:
from data import *
from model import *

import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset, train_test_loaders
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

In [2]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': False,
                               'tgt_tax_lvl': "species",
                               'fwd_only': True,
                               'manifold': True})

In [3]:
path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
chunks = LazySeqDataset(hparams, path=path,
                       keep_open=True)

In [32]:
dl0, dl1, dl2 = train_test_loaders(chunks, batch_size=16, distances=True)

In [5]:
#train_test_loaders??

In [34]:
#batch = next(iter(dl2))

In [6]:
#batch

In [7]:
def get_toy_dl(hparams, batch_size=16):
    path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
    chunks = LazySeqDataset(hparams, path=path,
                           keep_open=True)
    ds = taxon_ds(chunks, old_pad_seq)
    return DataLoader(ds, batch_size=batch_size, 
                      shuffle=True), ds

In [8]:
dl,ds = get_toy_dl(hparams, batch_size=128)
batch = next(iter(dl))
len(dl), batch[0].shape, batch[1].shape

(149, torch.Size([128, 1, 4096]), torch.Size([128]))

Let's creat a block class that we'll use for the inverted residual blocks

In [223]:
class DepSepBlock(nn.Module):
    def __init__(self):
        super(DepSepBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, 3)
        self.bn1 = nn.BatchNorm1d(32)
        self.act = nn.SiLU(inplace=True)
        self.conv2 = nn.Conv1d(32, 8, 1)
        self.conv3 = nn.Conv1d(8, 32, 1)
        self.conv4 = nn.Conv1d(32, 16, 1, stride=1)
        self.bn2 = nn.BatchNorm1d(16)
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.act(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.bn2(out)
        #out = nn.Identity(out)
        
        return out

In [216]:
dsc_out = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)(batch[0])
dsc_out.shape

torch.Size([128, 32, 2048])

In [225]:
DepSepBlock()(dsc_out).shape

torch.Size([128, 16, 2046])

In [213]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_ch:int, mid_ch:int, sq_ch:int,
                 out_ch:int, ks=1, stride: int = 1, padding: int =0):
        super(InvertedResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_ch, mid_ch, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm1d(mid_ch)
        self.act = nn.SiLU(inplace=True)
        self.conv2 = nn.Conv1d(mid_ch, mid_ch, kernel_size=ks, stride=stride,
                              padding=padding)
        self.bn2 = nn.BatchNorm1d(mid_ch)
        self.conv3 = nn.Conv1d(mid_ch, sq_ch, kernel_size=1,
                              stride=1) #squeeze excite conv
        self.conv4 = nn.Conv1d(sq_ch, mid_ch, kernel_size=1,
                              stride=1) #squeeze excite conv
        self.conv5 = nn.Conv1d(mid_ch, out_ch, kernel_size=1, stride=1) 
        self.bn3 = nn.BatchNorm1d(out_ch)
        
        self.downsample = nn.Conv1d(in_ch, out_ch,1, stride=stride)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.act(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.bn3(out)
        
        identity = self.downsample(x)
        out += identity
        out = self.act(out)
        
        return out

In [202]:
irb = InvertedResidualBlock(1, 5, 3, 6, 5, 2,2)
irb

InvertedResidualBlock(
  (conv1): Conv1d(1, 5, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
  (conv2): Conv1d(5, 5, kernel_size=(5,), stride=(2,), padding=(2,))
  (bn2): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(5, 3, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(3, 5, kernel_size=(1,), stride=(1,))
  (conv5): Conv1d(5, 6, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (downsample): Conv1d(1, 6, kernel_size=(1,), stride=(2,))
)

In [228]:
irb(batch[0]).shape

torch.Size([128, 6, 2048])

In [277]:
class TestNet(nn.Module):
    def __init__(self, param_list, avg_out=200, out_feats=512):
        super(TestNet, self).__init__()

        self.avg_out = avg_out
        self.out_feats = out_feats
        self.param_list = param_list


        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm1d(32)
        self.silu = nn.SiLU(inplace=True)
        self.layer0 = DepSepBlock()
        self.layer1 = self._make_layer(self.param_list[0])
        self.layer2 = self._make_layer(self.param_list[1])
        self.layer3 = self._make_layer(self.param_list[2])
        self.layer4 = self._make_layer(self.param_list[3])
        self.layer5 = self._make_layer(self.param_list[4])
        self.layer6 = self._make_layer(self.param_list[5])
        self.conv2 = nn.Conv1d(320, 1280, 1, 1)
        self.bn2 = nn.BatchNorm1d(1280)
        self.avgpool = nn.AdaptiveAvgPool1d(output_size=self.avg_out)
        self.fc = nn.Linear(in_features=self.avg_out, 
                            out_features=self.out_feats)
    
    def _make_layer(self, param_list):
        layers = []
        for x in range(len(param_list)):
            layers.append(InvertedResidualBlock(*param_list[x]))
        #layers.append(InvertedResidualBlock(32,5,6,7))
        #layers.append(InvertedResidualBlock(7,8,9,10))
        return nn.Sequential(*layers)
        
    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.silu(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
    def forward(self, x):
        return self._forward_impl(x)

In [265]:
#TestNet(p_list)

In [278]:
p_list = [
        [[16,96,3,24,3,2,1], [24,144,6,24,3,1,1]], #layer 1
        [[24,144,6,40,5,2,2], [40,240,10,40,5,1,2]], #layer 2 
        [[40,240,10,80,3,2,1],[80,480,20,80,3,1,1], 
                                [80,480,20,80,3,1,1]], #layer 3
        [[80,480,20,112,5,1,2], [112,672,28,112,5,1,2], 
                                 [112,672,28,112,5,1,2]], #layer 4
        [[112,672,28,192,5,2,2],[192,1152,48,192,5,1,2],
         [192,1152,48,192,5,1,2], [192,1152,48,192,5,1,2]], #layer 5
        [[192,1152,48,320,3,2,1]] #layer 6
         ]

In [269]:
#672/112

In [270]:
#InvertedResidualBlock(*[16, 96, 24, 4, 3, 2, 1])

In [279]:
out = TestNet(p_list)(batch[0])
out.shape

torch.Size([128, 1280, 512])

In [272]:
TestNet(p_list)

TestNet(
  (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (silu): SiLU(inplace=True)
  (layer0): DepSepBlock(
    (conv1): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
    (conv2): Conv1d(32, 8, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(8, 32, kernel_size=(1,), stride=(1,))
    (conv4): Conv1d(32, 16, kernel_size=(1,), stride=(1,))
    (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer1): Sequential(
    (0): InvertedResidualBlock(
      (conv1): Conv1d(16, 96, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
      (conv2): Conv1d(96, 96, kernel_size=(3,), stride=(2,), padding=(1,))
      (bn2)

In [27]:
def get_eff_b0(out_feats):
    return nn.Sequential(
        get_base_layer(),
        get_dep_sep(32, 16),

        #layer 1 has two inverted residuals (IRs)
        get_inv_res(in_ch=16, mid_ch=96, out_ch=24,
                   sq_ch=4, ks=3, stride=2, padding=1),
        get_inv_res(in_ch=24, mid_ch=144, out_ch=24,
                   sq_ch=6, ks=3, stride=1, padding=1),

        #layers 2 has 2 IR's
        get_inv_res(in_ch=24, mid_ch=144, out_ch=40,
                   sq_ch=6, ks=5, stride=2, padding=2),
        get_inv_res(in_ch=40, mid_ch=240, out_ch=40,
                   sq_ch=10, ks=5, stride=2, padding=2),

        #layer 3 has 3 IR's
        get_inv_res(in_ch=40, mid_ch=240, out_ch=80,
                   sq_ch=10, ks=3, stride=2, padding=1),
        get_inv_res(in_ch=80, mid_ch=480, out_ch=80,
                   sq_ch=20, ks=3, stride=1, padding=1),
        get_inv_res(in_ch=80, mid_ch=480, out_ch=80,
                   sq_ch=20, ks=3, stride=1, padding=1),

        #layer 4 has 3 inverted residuals
        get_inv_res(in_ch=80, mid_ch=480, out_ch=112,
                   sq_ch=20, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=112, mid_ch=672, out_ch=112,
                   sq_ch=28, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=112, mid_ch=672, out_ch=112,
                   sq_ch=28, ks=5, stride=1, padding=2),

        #layer 5 has 4 inverted residuals
        get_inv_res(in_ch=112, mid_ch=672, out_ch=192,
                   sq_ch=28, ks=5, stride=2, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        #layer 6 has 1 IR
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=320,
                   sq_ch=48, ks=3, stride=1, padding=1),

        get_head_layer(out_chans=out_feats)
    )

In [32]:
%time
effnet_b0(batch[0]).shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


torch.Size([128, 512, 1])

In [37]:
effnet_b0.to('cuda');
#batch[0].to('cuda');

In [38]:
%time
effnet_b0(batch[0].to('cuda')).shape

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


torch.Size([128, 512, 1])

In [30]:
effnet_b0

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), bias=False)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv1d(32, 8, kernel_size=(2,), stride=(2,))
      (1): SiLU()
      (2): Conv1d(8, 32, kernel_size=(2,), stride=(2,))
    )
    (2): Sequential(
      (0): Conv1d(32, 16, kernel_size=(1,), stride=(1,), bias=False)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Identity()
  )
  (2): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(16, 96, kernel_